In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134"}

In [ ]:
#url = 'https://listings.offcampusliving.cornell.edu/listings?search=&priceMin=500&priceMax=5000&bedroom=10&pets=any&likes=false&view=list&safety=false'

In [7]:
try:
    with open('Listings.html', 'r', encoding='utf-8') as file:
        file_content = file.read()
except Exception as e:
    print('Failed to read the file:', e)

In [8]:
soup = BeautifulSoup(file_content, 'html.parser')

In [10]:
listings = soup.find_all('mat-card', class_='mat-mdc-card mdc-card')
listings

[<mat-card _ngcontent-ng-c493594249="" class="mat-mdc-card mdc-card"><div _ngcontent-ng-c493594249="" class="card-content-wrapper"><div _ngcontent-ng-c493594249="" class="card-image-wrapper"><img _ngcontent-ng-c493594249="" alt="Photo of 222-26 EDDY STREET" class="image-loaded" default="/assets/placeholder.jpg" src="./Listings_files/222 Eddy Front.jpg"/></div><div _ngcontent-ng-c493594249="" class="card-details"><mat-card-header _ngcontent-ng-c493594249="" class="mat-mdc-card-header"><div class="mat-mdc-card-header-text"></div><mat-card-title-group _ngcontent-ng-c493594249="" class="mat-mdc-card-title-group"><div><mat-card-title _ngcontent-ng-c493594249="" class="mat-mdc-card-title">222-26 EDDY STREET</mat-card-title><mat-card-subtitle _ngcontent-ng-c493594249="" class="mat-mdc-card-subtitle">Bedroom available in 7 Bedroom 2 Bath Townhouse- Spring 2024 </mat-card-subtitle></div></mat-card-title-group></mat-card-header><mat-card-content _ngcontent-ng-c493594249="" class="mat-mdc-card-co

In [24]:
extracted_records = []

for listing in listings:
    try:
        # Extract the title, subtitle, and image as before
        title = listing.find('mat-card-title', class_='mat-mdc-card-title').get_text(strip=True)
        subtitle = listing.find('mat-card-subtitle', class_='mat-mdc-card-subtitle').get_text(strip=True)
        image_element = listing.find('img')
        image_url = image_element['src'] if image_element else None
        more_details_link = listing.find('a', {'mat-stroked-button': True})['href']

        details_elements = listing.find_all('div', class_='detail')
        
        # Create a dictionary to hold the parsed details
        details = {}
        for element in details_elements:
            # Ignore icon elements to avoid including their text
            for icon in element.find_all('mat-icon', class_='mat-icon'):
                icon.decompose()  # Remove this element from the parse tree

            # The text of the element now doesn't contain the icon text
            detail_text = element.get_text(strip=True)

            # Split the detail by ':' to separate field name and value
            detail_parts = detail_text.split(':', 1)  # Split only at the first ':'

            if len(detail_parts) == 2:
                detail_name, detail_value = detail_parts
                detail_name = detail_name.strip()
                detail_value = detail_value.strip()

                # Store detail in the dictionary
                details[detail_name] = detail_value

        # Construct the record for this listing
        record = {
            'title': title,
            'subtitle': subtitle,
            'image_url': image_url,
            'details': details,
            'more_details_link': more_details_link,
        }
        extracted_records.append(record)

    except Exception as e:
        print(f"Error parsing listing: {e}")


extracted_records

[{'title': '222-26 EDDY STREET',
  'subtitle': 'Bedroom available in 7 Bedroom 2 Bath Townhouse- Spring 2024',
  'image_url': './Listings_files/222 Eddy Front.jpg',
  'details': {'Safety RatingCompliant': 'COC Expires 12/28/24Info about safety ratings',
   'RentAmount': '$895',
   'Date Available': 'Dec 31, 2023',
   'HousingType': 'Rent',
   'Bedrooms': 'Beds:1',
   'Bathrooms': 'Baths:2',
   'Pets': 'No'},
  'more_details_link': 'https://listings.offcampusliving.cornell.edu/listings/view/118'},
 {'title': '112-14 FERRIS PLACE',
  'subtitle': '1 bdrm in renovated house available for spring semester',
  'image_url': './Listings_files/new kitchen pic 3.JPG',
  'details': {'Safety RatingCompliant': 'COC Expires 8/30/25Info about safety ratings',
   'RentAmount': '$800',
   'Date Available': 'Jan 7, 2024',
   'HousingType': 'Room to Rent',
   'Bedrooms': 'Beds:6',
   'Bathrooms': 'Baths:2',
   'Pets': 'No'},
  'more_details_link': 'https://listings.offcampusliving.cornell.edu/listings/vie

In [27]:
df = pd.DataFrame(extracted_records)
df

,title,subtitle,image_url,details,more_details_link
0,222-26 EDDY STREET,Bedroom available in 7 Bedroom 2 Bath Townhous...,./Listings_files/222 Eddy Front.jpg,{'Safety RatingCompliant': 'COC Expires 12/28/...,https://listings.offcampusliving.cornell.edu/l...
1,112-14 FERRIS PLACE,1 bdrm in renovated house available for spring...,./Listings_files/new kitchen pic 3.JPG,{'Safety RatingCompliant': 'COC Expires 8/30/2...,https://listings.offcampusliving.cornell.edu/l...
2,410 STEWART AVENUE,3 Bedroom Apartment for rent 2024-2025,./Listings_files/placeholder.jpg,{'Safety RatingCompliant': 'COC Expires 1/7/26...,https://listings.offcampusliving.cornell.edu/l...
3,414 STEWART AVENUE,TOP FLOOR APARTMENT - Great view and two block...,./Listings_files/414 building.PNG,{'Safety RatingCompliant': 'COC Expires 5/30/2...,https://listings.offcampusliving.cornell.edu/l...
4,312-24 THURSTON AVENUE,3 bedroom 2 bath,./Listings_files/c14 living room.JPG,{'Safety RatingCompliant': 'COC Expires 6/9/24...,https://listings.offcampusliving.cornell.edu/l...
...,...,...,...,...,...
95,305 Eddy Street,seeking subletters for 2 rooms in 3 bedroom ap...,./Listings_files/C7AC38FE-90A7-427B-BC4F-F3363...,{'Safety RatingCompliant': 'COC Expires 4/12/2...,https://listings.offcampusliving.cornell.edu/l...
96,318-20 Pleasant Street,Seeking a sublease/lease transfer for a 3b/2b ...,./Listings_files/IMG_5774.jpg,{'Safety RatingCompliant': 'COC Expires 11/11/...,https://listings.offcampusliving.cornell.edu/l...
97,937 State Street E,"Jan-May 2024 sublease: Fully furnished 1 bed, ...",./Listings_files/dining.jpg,{'Safety RatingCompliant': 'COC Expires 5/26/2...,https://listings.offcampusliving.cornell.edu/l...
98,303 E Court,1 Bed / 1 Bath with bonus room/office. In dow...,./Listings_files/Front Room.jpg,"{'RentAmount': '$1530', 'Date Available': 'Oct...",https://listings.offcampusliving.cornell.edu/l...
